# Train/Dev/Test split of AotM-2011 songs in setting I

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys
import gzip
import pickle as pkl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score
from scipy.optimize import check_grad
from scipy.sparse import lil_matrix, issparse
from collections import Counter
import itertools as itt

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sys.path.append('src')
from BinaryRelevance import BinaryRelevance
#from PClassificationMLC import PClassificationMLC
from PCMLC import PCMLC as PClassificationMLC
from PCMLC import obj_pclassification
from evaluate import calc_F1, calc_precisionK, calc_rank, f1_score_nowarn, evalPred

In [ ]:
data_dir = 'data/aotm-2011'
#faotm = os.path.join(data_dir, 'aotm2011-subset.pkl')
faotm = os.path.join(data_dir, 'aotm2011-user-playlist.pkl')
#ffeature = 'data/msd/songID2Features.pkl.gz'
ffeature = 'data/msd/song2feature.pkl.gz'
fgenre = 'data/msd/song2genre.pkl'

## Load playlists

Load playlists.

In [ ]:
user_playlists = pkl.load(open(faotm, 'rb'))

In [ ]:
all_users = sorted(user_playlists.keys())

In [ ]:
all_playlists = [(pl, u) for u in all_users for pl in user_playlists[u]]

In [ ]:
print('#user    : {:,}'.format(len(all_users)))
print('#playlist: {:,}'.format(len(all_playlists)))

In [ ]:
pl_lengths = [len(pl) for u in all_users for pl in user_playlists[u]]
#plt.hist(pl_lengths, bins=100)
print('Average playlist length: %.1f' % np.mean(pl_lengths))

## Load song features

Load `song_id` --> `feature array` mapping: map a song to the audio features of one of its corresponding tracks in MSD.

In [ ]:
song2feature = pkl.load(gzip.open(ffeature, 'rb'))

## Split songs for setting I

Split songs (70/10/20 train/dev/test split) the latest released (year) songs are in dev and test set.

In [ ]:
all_songs = sorted([(sid, song2feature[sid][-1]) for sid in \
                    {s for u in all_users for pl in user_playlists[u] for s in pl}], key=lambda x: x[1])
song_indices = {sid: ix for ix, (sid, _) in enumerate(all_songs)}
print(len(all_songs))
#song_set

In [ ]:
nsong_dev_test = int(len(all_songs) * 0.3)
train_song_set = all_songs[nsong_dev_test:]

# shuffle songs in dev and test set
np.random.seed(123456789)
dev_test_ix = np.random.permutation(np.arange(nsong_dev_test))
nsong_dev = int(len(all_songs) * 0.1)
dev_song_set = [all_songs[ix] for ix in dev_test_ix[:nsong_dev]]
test_song_set = [all_songs[ix] for ix in dev_test_ix[nsong_dev:]]

In [ ]:
print('#songs in training set: {:,}, average song age: {:.2f} yrs'
      .format(len(train_song_set), np.mean([t[1] for t in train_song_set])))
print('#songs in dev set     : {:,}, average song age: {:.2f} yrs'
      .format(len(dev_song_set),   np.mean([t[1] for t in dev_song_set])))
print('#songs in test set    : {:,}, average song age: {:.2f} yrs'
      .format(len(test_song_set),  np.mean([t[1] for t in test_song_set])))

In [ ]:
print('{:,} | {:,}'.format(len(all_songs), len({s for s in train_song_set + dev_song_set + test_song_set})))

Song popularity.

In [ ]:
song_pl_mat = np.zeros((len(all_songs), len(all_playlists)), dtype=np.int8)
for j in range(len(all_playlists)):
    pl = all_playlists[j][0]
    ind = [song_indices[sid] for sid in pl]
    song_pl_mat[ind, j] = 1

In [ ]:
song_pop = np.sum(song_pl_mat, axis=1)

In [ ]:
ax = plt.subplot(111)
ax.hist(song_pop, bins=100)
ax.set_yscale('log')
ax.set_title('Histogram of song popularity')
pass

In [ ]:
train_song_pop = [song_pop[song_indices[sid]] for (sid, _) in train_song_set]
ax = plt.subplot(111)
ax.hist(train_song_pop, bins=100)
ax.set_yscale('log')
ax.set_xlim(0, song_pop.max()+1)
ax.set_title('Histogram of song popularity in TRAINING set')
pass

In [ ]:
dev_song_pop = [song_pop[song_indices[sid]] for (sid, _) in dev_song_set]
ax = plt.subplot(111)
ax.hist(dev_song_pop, bins=100)
ax.set_yscale('log')
ax.set_xlim(0, song_pop.max()+1)
ax.set_title('Histogram of song popularity in DEV set')
pass

In [ ]:
test_song_pop = [song_pop[song_indices[sid]] for (sid, _) in test_song_set]
ax = plt.subplot(111)
ax.hist(test_song_pop, bins=100)
ax.set_yscale('log')
ax.set_xlim(0, song_pop.max()+1)
ax.set_title('Histogram of song popularity in TEST set')
pass

## Load genres

Song genres from [MSD Allmusic Genre Dataset (Top MAGD)](http://www.ifs.tuwien.ac.at/mir/msd/TopMAGD.html) and [tagtraum](http://www.tagtraum.com/msd_genre_datasets.html).

In [ ]:
song2genre = pkl.load(open(fgenre, 'rb'))

Check if all songs have genre info.

In [ ]:
print('#songs missing genre: {:,}'.format(len(all_songs) - np.sum([sid in song2genre for (sid, _) in all_songs])))

## Create song-playlist matrix

Songs as rows, playlists as columns.

In [ ]:
def gen_dataset(playlists, song2feature, song2genre, train_song_set, dev_song_set=[], test_song_set=[]):
    """
    Create labelled dataset: rows are songs, columns are users.
    
    Input:
        - playlists: a set of playlists
        - train_song_set: a list of songIDs in training set
        - dev_song_set: a list of songIDs in dev set
        - test_song_set: a list of songIDs in test set
        - song2feature: dictionary that maps songIDs to features from MSD
        - song2genre: dictionary that maps songIDs to genre
    Output:
        - (Feature, Label) pair (X, Y)
          X: #songs by #features
          Y: #songs by #users
    """ 
    song_set = train_song_set + dev_song_set + test_song_set
    N = len(song_set)
    K = len(playlists)
    
    genre_set = sorted({v for v in song2genre.values()})
    genre2index = {genre: ix for ix, genre in enumerate(genre_set)}
    
    def onehot_genre(songID):
        """
        One-hot encoding of genres.
        Data imputation: 
            - one extra entry for songs without genre info
            - mean imputation
            - sampling from the distribution of genre popularity
        """
        num = len(genre_set) # + 1
        vec = np.zeros(num, dtype=np.float)
        if songID in song2genre:
            genre_ix = genre2index[song2genre[songID]]
            vec[genre_ix] = 1
        else:
            vec[:] = np.nan
            #vec[-1] = 1
        return vec
    
    #X = np.array([features_MSD[sid] for sid in song_set])  # without using genre
    #Y = np.zeros((N, K), dtype=np.bool)
    X = np.array([np.concatenate([song2feature[sid], onehot_genre(sid)], axis=-1) for sid in song_set])
    Y = lil_matrix((N, K), dtype=np.bool)
    
    song2index = {sid: ix for ix, sid in enumerate(song_set)}
    for k in range(K):
        pl = playlists[k]
        indices = [song2index[sid] for sid in pl if sid in song2index]
        Y[indices, k] = True
        
    # genre imputation
    genre_ix_start = -len(genre_set)
    genre_nan = np.isnan(X[:, genre_ix_start:])
    genre_mean = np.nansum(X[:, genre_ix_start:], axis=0) / (X.shape[0] - np.sum(genre_nan, axis=0))
    #print(np.nansum(X[:, genre_ix_start:], axis=0))
    #print(genre_set)
    #print(genre_mean)
    for j in range(len(genre_set)):
        X[genre_nan[:,j], j+genre_ix_start] = genre_mean[j]

    #return X, Y
    Y = Y.tocsr()
    
    train_ix = [song2index[sid] for sid in train_song_set]
    X_train = X[train_ix, :]
    Y_train = Y[train_ix, :]
    
    dev_ix = [song2index[sid] for sid in dev_song_set]
    X_dev = X[dev_ix, :]
    Y_dev = Y[dev_ix, :]
    
    test_ix = [song2index[sid] for sid in test_song_set]
    X_test = X[test_ix, :]
    Y_test = Y[test_ix, :]
    
    if len(dev_song_set) > 0:
        if len(test_song_set) > 0:
            return X_train, Y_train, X_dev, Y_dev, X_test, Y_test
        else:
            return X_train, Y_train, X_dev, Y_dev
    else:
        if len(test_song_set) > 0:
            return X_train, Y_train, X_test, Y_test
        else:
            return X_train, Y_train

## Setting I: hold a subset of songs, use all playlists

In [ ]:
pkl_dir = os.path.join(data_dir, 'setting1')
fxtrain = os.path.join(pkl_dir, 'X_train.pkl.gz')
fytrain = os.path.join(pkl_dir, 'Y_train.pkl.gz')
fxdev   = os.path.join(pkl_dir, 'X_dev.pkl.gz')
fydev   = os.path.join(pkl_dir, 'Y_dev.pkl.gz')
fxtest  = os.path.join(pkl_dir, 'X_test.pkl.gz')
fytest  = os.path.join(pkl_dir, 'Y_test.pkl.gz')
fadjmat = os.path.join(pkl_dir, 'adjacency_mat.pkl.gz')

In [ ]:
X_train, Y_train, X_dev, Y_dev, X_test, Y_test = gen_dataset(playlists = [t[0] for t in all_playlists],
                                                             song2feature = song2feature, song2genre = song2genre,
                                                             train_song_set = [t[0] for t in train_song_set],
                                                             dev_song_set   = [t[0] for t in   dev_song_set], 
                                                             test_song_set  = [t[0] for t in  test_song_set])

Feature normalisation.

In [ ]:
X_train_mean = np.mean(X_train, axis=0).reshape((1, -1))
X_train_std = np.std(X_train, axis=0).reshape((1, -1)) + 10 ** (-6)
X_train -= X_train_mean
X_train /= X_train_std
X_dev   -= X_train_mean
X_dev   /= X_train_std
X_test  -= X_train_mean
X_test  /= X_train_std

In [ ]:
print('Train: %15s %15s' % (X_train.shape, Y_train.shape))
print('Dev  : %15s %15s' % (X_dev.shape,   Y_dev.shape))
print('Dev  : %15s %15s' % (X_dev.shape,   Y_dev.shape))

In [ ]:
print(np.mean(np.mean(X_train, axis=0)))
print(np.mean( np.std(X_train, axis=0)) - 1)
print(np.mean(np.mean(X_dev,   axis=0)))
print(np.mean( np.std(X_dev,   axis=0)) - 1)
print(np.mean(np.mean(X_test,  axis=0)))
print(np.mean( np.std(X_test,  axis=0)) - 1)

In [ ]:
pkl.dump(X_train, gzip.open(fxtrain, 'wb'))
pkl.dump(Y_train, gzip.open(fytrain, 'wb'))
pkl.dump(X_dev,   gzip.open(fxdev,   'wb'))
pkl.dump(Y_dev,   gzip.open(fydev,   'wb'))
pkl.dump(X_test,  gzip.open(fxtest,  'wb'))
pkl.dump(Y_test,  gzip.open(fytest,  'wb'))

In [ ]:
user_of_playlists = [t[1] for t in all_playlists]
npl = len(user_of_playlists)
same_user_mat = lil_matrix((npl, npl), dtype=np.bool)
for u in set(user_of_playlists):
    clique = np.where(u == np.array(user_of_playlists, dtype=np.object))[0]
    for x, y in itt.combinations(clique, 2):
        same_user_mat[x, y] = True
        same_user_mat[y, x] = True

In [ ]:
#same_user_mat

In [ ]:
pkl.dump(same_user_mat.tocsr(), gzip.open(fadjmat, 'wb'))

## Setting II

### Split playlists

Split playlists (80/20 split) uniformly at random ~~such that the distributions of playlist length (the number of songs in playlists) for each user in training and dev set are similiar~~.

In [ ]:
train_playlists = []
dev_playlists = []

In [ ]:
ratio = 0.2
np.random.seed(123456789)
for u in uid_subset:
    for pl in user_playlists[u]:
        if np.random.rand() < 0.2:
            dev_playlists.append((u, pl))
        else:
            train_playlists.append((u, pl))

In [ ]:
print(len(train_playlists), len(dev_playlists))

In [ ]:
#ratio = 0.2
#step = 1./ratio
#np.arange(0, 10, step)
#np.random.seed(123456789)
#rounding_prob = step - int(step)
#for u in uid_subset:
#    u_playlists = user_playlists[u]
#    if len(u_playlists) < 3: 
#        train_playlists.append((u, u_playlists[0]))
#        continue
#    sorted_pl = sorted(u_playlists, key=lambda pl: len(pl))
#    if step == int(step):
#        step = int(step)
#        dev_ix = np.arange(0, len(sorted_pl), step)
#    else:
#        split_ix = np.arange(0, len(sorted_pl), step)
#        dev_ix = [ix for ix in [int(x) if np.random.rand() < rounding_prob or int(x) == len(sorted_pl)-1 \
#                                else int(x)+1 for x in split_ix]]  # avoid index out of bounds
#    dev_playlists += [(u, sorted_pl[ix]) for ix in dev_ix]
#    train_playlists += [(u, sorted_pl[ix]) for ix in range(len(sorted_pl)) if ix not in dev_ix]

In [ ]:
xmax = np.max([len(pl) for pl in playlists_subset]) + 1

Histogram of playlist length in training set.

In [ ]:
ax = plt.subplot(111)
ax.hist([len(t[1]) for t in train_playlists], bins=50)
ax.set_yscale('log')
ax.set_xlim(0, xmax)
print(len(train_playlists))

Histogram of playlist length in training set.

In [ ]:
ax = plt.subplot(111)
ax.hist([len(t[1]) for t in dev_playlists], bins=50)
ax.set_yscale('log')
ax.set_xlim(0, xmax)
print(len(dev_playlists))

### Hold part of songs in the dev set of playlists

Hold the last half of songs for each playlist in dev set.

In [ ]:
dev_playlists_obs = [pl[:-int(len(pl)/2)] for (_, pl) in dev_playlists]
dev_playlists_held = [pl[-int(len(pl)/2):] for (_, pl) in dev_playlists]

In [ ]:
for i in range(len(dev_playlists)):
    assert np.all(dev_playlists[i][1] == dev_playlists_obs[i] + dev_playlists_held[i])

In [ ]:
print('obs: %d, held: %d' % (np.sum([len(ppl) for ppl in dev_playlists_obs]), 
                             np.sum([len(ppl) for ppl in dev_playlists_held])))

In [ ]:
#ratio = 0.1
#np.random.seed(987654321)
#num_held = 0
#for u, pl in dev_playlists:
#    sample_size = ratio * len(pl)
#    rounding_prob = sample_size - int(sample_size)
#    sample_size = int(sample_size) if np.random.rand() < rounding_prob else int(sample_size) + 1
#    sample_ix = np.random.permutation(np.arange(len(pl)))[sample_size:]
#    dev_known_songs += np.array(pl)[sample_ix].tolist()
#    num_held += sample_size
#print('#song being held:', num_held)

In [ ]:
def mean_normalised_reciprocal_rank(Y_true, Y_pred):
    """
    Compute the mean of normalised reciprocal rank (reciprocal rank are normalised by the best possible ranks)
    """
    normalised_reci_rank = []
    npos = np.sum(Y_true, axis=0)
    for k in range(Y_true.shape[1]):
        ranks = calc_rank(Y_pred[:, k])[Y_true[:, k]]
        if len(ranks) > 0:
            ideal = np.sum([1./nk for nk in range(1, npos[k]+1)])
            real = np.sum([1./r for r in ranks])
            normalised_reci_rank.append(real / ideal)  # normalise the reciprocal ranks by the best possible ranks
    return np.mean(normalised_reci_rank)

In [ ]:
def eval_pl(Y_true, Y_pred, top=100, useLoop=False):
    if useLoop is False:
        assert Y_true.shape == Y_pred.shape
        assert top <= Y_true.shape[0]
        nzcol = np.nonzero(np.sum(Y_true, axis=0))[0]  # columns with at least one True
        ncols = len(nzcol)
        topix = np.argsort(-Y_pred, axis=0)[:top, :]
        npos = np.sum(Y_true, axis=0)
        hr = np.mean([np.sum(Y_true[topix[:, j], j]) / npos[j] for j in nzcol])
        paks, valid_indices = calc_precisionK(Y_true.T, Y_pred.T)
        pak = np.mean(paks[valid_indices])
        auc = roc_auc_score(Y_true[:, nzcol], Y_pred[:, nzcol], average='macro')
        ap  = average_precision_score(Y_true[:, nzcol], Y_pred[:, nzcol], average='macro')
        nrr = mean_normalised_reciprocal_rank(Y_true, Y_pred)
    else:
        assert type(Y_true) == list
        assert type(Y_pred) == list
        assert len(Y_true) == len(Y_pred)
        hitrates, paks, aucs, aps, nrrs = [], [], [], [], []
        for j in range(len(Y_true)):
            if np.sum(Y_true[j]) < 1: continue   # filtering out cases where all ground truths are negative.
            gt = Y_true[j].reshape(-1)
            pred = Y_pred[j].reshape(-1)
            assert gt.shape == pred.shape
            assert top <= gt.shape[0]
            topix = np.argsort(-pred)[:top]
            hitrates.append(np.sum(gt[topix]) / np.sum(gt))
            #paks.append(calc_precisionK(gt.reshape(1,-1), pred.reshape(1,-1)))  # incorrect
            paks.append(evalPred(gt, pred, metricType='Precision@K'))
            aucs.append(roc_auc_score(gt, pred))
            aps.append(average_precision_score(gt, pred))
            nrrs.append(mean_normalised_reciprocal_rank(gt.reshape(-1,1), pred.reshape(-1,1)))
        hr, pak, auc, ap, nrr = [np.mean(x) for x in [hitrates, paks, aucs, aps, nrrs]]
        ncols = len(paks)
      
    print('Average over %d columns' % ncols)
    print('%-20s %.4f' % ('Mean HitRate@100:', hr))
    print('%-20s %.4f' % ('Mean P@K:', pak))
    print('%-20s %.4f' % ('Mean AUC:', auc))
    print('%-20s %.4f' % ('MAP:', ap))
    print('%-20s %.4f' % ('Mean NRR:', nrr))

Build the adjacent matrix of playlists (nodes), playlists of the same user form a *clique*.

In [ ]:
user_of_playlists2 = [t[0] for t in train_playlists + dev_playlists]
#user_of_playlists2

In [ ]:
npl = len(user_of_playlists2)
same_user_mat = np.zeros((npl, npl), dtype=np.bool)
for i in range(npl):
    for j in range(i+1, npl):
        if user_of_playlists2[i] == user_of_playlists2[j]:
            same_user_mat[i, j] = True
            same_user_mat[j, i] = True
#same_user_mat

Check gradient.

In [ ]:
%%script false
w0 = 0.001 * np.random.randn(Y_dev.shape[1] * X_dev.shape[1] + 1)
check_grad(lambda w: obj_pclassification(w, X_dev, Y_dev, p=1, C1=2, C2=3, C3=5,
                                         weighting='samples', similarMat=same_user_mat)[0], 
           lambda w: obj_pclassification(w, X_dev, Y_dev, p=1, C1=2, C2=3, C3=5,
                                         weighting='samples', similarMat=same_user_mat)[1], w0)

In [ ]:
#np.sum(Y_train, axis=0)

In [ ]:
#np.sum(Y_dev, axis=0)

### M1. BR - Independent logistic regression

In [ ]:
br = BinaryRelevance(C=1, n_jobs=4)
br.fit(X_train, Y_train)

Evaluation: normalise **per playlist**.

In [ ]:
print('Dev set:')
eval_pl(Y_dev, br.predict(X_dev))

In [ ]:
print('Training set:')
eval_pl(Y_train, br.predict(X_train))

### M2. PC - Multilabel p-classification

P-Classification ~ P-norm push ranking.

In [ ]:
pc = PClassificationMLC(C1=1, C2=1, C3=10, weighting='both', similarMat=same_user_mat)
pc.fit(X_train, Y_train)

Evaluation: normalise **per playlist**.

In [ ]:
print('Dev set:')
eval_pl(Y_dev, pc.predict(X_dev))

In [ ]:
print('Dev set:')
gts = [Y_dev[:,k] for k in range(Y_dev.shape[1])]
Y_pred = pc.predict(X_dev)
preds = [Y_pred[:,k] for k in range(Y_pred.shape[1])]
eval_pl(gts, preds, top=100, useLoop=True)

In [ ]:
print('Training set:')
eval_pl(Y_train, pc.predict(X_train))

**Performance per user**

In [ ]:
user_of_playlists2 = [t[0] for t in train_playlists + dev_playlists]
user_set = sorted(set(user_of_playlists2))
Y_pc_train = pc.predict(X_train)
Y_pc_dev = pc.predict(X_dev)
for u in user_set:
    uind = np.where(np.array(user_of_playlists2, dtype=np.object) == u)[0]
    if len(uind) < 2: continue
    print('--------------------')
    print('USER:', u)
    print('#playlist: %d' % len(uind))
    print('Dev set:')
    eval_pl(Y_dev[:, uind], Y_pc_dev[:, uind])
    print('\nTraining set:')
    eval_pl(Y_train[:, uind], Y_pc_train[:, uind])
    print()

## Setting II: hold a subset of songs in a subset of playlists, use all songs

In [ ]:
X, Y = gen_dataset_subset(playlists=[t[1] for t in train_playlists + dev_playlists], 
                          song_set=[t[0] for t in train_song_set + dev_song_set], 
                          song2feature=song2feature, song2genre=song2genre)

In [ ]:
dev_cols = np.arange(len(train_playlists), Y.shape[1])
col_split = len(train_playlists)

Set all entries corresponding to playlists in dev set to NaN, except those songs in dev playlists that we observed.

In [ ]:
Y_train = Y.copy().astype(np.float)  # note: np.nan is float
Y_train[:, dev_cols] = np.nan
song_indices = {sid: ix for ix, (sid, _) in enumerate(song_set)}
assert len(dev_cols) == len(dev_playlists) == len(dev_playlists_obs)
num_known = 0
for j in range(len(dev_cols)):
    rows = [song_indices[sid] for sid in dev_playlists_obs[j]]
    Y_train[rows, dev_cols[j]] = 1
    num_known += len(rows)

In [ ]:
isnanmat = np.isnan(Y_train)

In [ ]:
print('#unknown: {:,} {:,}'.format(np.sum(isnanmat), len(dev_playlists) * len(song_set) - num_known))

In [ ]:
print('#unknown in setting I: {:,}'.format(len(dev_song_set) * Y.shape[1]))

In [ ]:
print(np.sum(isnanmat[:, :col_split]))  # number of NaN in training playlists, should be 0

In [ ]:
X_train = X

Feature normalisation.

In [ ]:
X_train_mean = np.mean(X_train, axis=0).reshape((1, -1))
X_train_std = np.std(X_train, axis=0).reshape((1, -1)) + 10 ** (-6)
X_train -= X_train_mean
X_train /= X_train_std

In [ ]:
print('Train: %15s %15s' % (X_train.shape, Y_train.shape))

In [ ]:
print(np.mean(np.mean(X_train, axis=0)))
print(np.mean( np.std(X_train, axis=0)) - 1)

In [ ]:
#np.sum(Y_train, axis=0)

### M3. Independent logistic regression

In [ ]:
br2 = BinaryRelevance(C=1, n_jobs=4)
br2.fit(X_train, np.nan_to_num(Y_train))

Evaluation: normalise **per playlist**.

In [ ]:
gts = [Y[isnanmat[:, col], col] for col in dev_cols]

In [ ]:
print('Dev set:')
Y_br2 = br2.predict(X_train)
preds = [Y_br2[isnanmat[:, col], col] for col in dev_cols]
eval_pl(gts, preds, useLoop=True)

In [ ]:
print('Training set:')
col_split = len(train_playlists)
eval_pl(Y_train[:, :col_split].astype(np.bool), br2.predict(X_train)[:, :col_split])

### M4. Multilabel p-classification with some playlist fully observed

In [ ]:
pla = PClassificationMLC(C1=1, C2=1, C3=10, weighting='both', similarMat=same_user_mat)
pla.fit(X_train, Y_train)

Evaluation: normalise **per playlist**.

In [ ]:
print('Dev set:')
Y_pla = pla.predict(X_train)
preds = [Y_pla[isnanmat[:, col], col] for col in dev_cols]
eval_pl(gts, preds, useLoop=True)

In [ ]:
print('Training set:')
col_split = len(train_playlists)
eval_pl(Y_train[:, :col_split].astype(np.bool), pla.predict(X_train)[:, :col_split])

**Check the if the regulariser is effective**

In [ ]:
%%script false
rows, cols = np.nonzero(same_user_mat)
for row, col in zip(rows, cols):
    diff = pla.W[row] - pla.W[col]
    print('%g' % np.sqrt(np.dot(pla.W[row], pla.W[row])))
    print('%g' % np.sqrt(np.dot(pla.W[col], pla.W[col])))
    print('%g' % np.sqrt(np.dot(diff, diff)))
    print('------------------------------')

Compute matrix $M$ such that $M_{jk} = \sqrt{(w_j - w_k)^\top (w_j - w_k)}, \forall j, k$.

In [ ]:
A = np.dot(pla.W, pla.W.T)
B = np.tile(np.diag(A), (A.shape[0], 1))
M = np.sqrt(-2 * A + (B + B.T))

Normalise $M$ by the vector with maximum norm in $W$.

In [ ]:
#aa = np.arange(6).reshape(3, 2)
#np.einsum('ij,ij->i', aa, aa)

In [ ]:
denorm = np.sqrt(np.einsum('ij,ij->i', pla.W, pla.W))  # compute the norm for each row in W

In [ ]:
M1 = M / np.max(denorm)

In [ ]:
plt.matshow(M1)

In [ ]:
rows, cols = np.nonzero(same_user_mat)
M2 = M1[rows, cols]
print('Min: %.5f, Max: %.5f, Mean: %.5f, Std: %.5f' % (np.min(M2), np.max(M2), np.mean(M2), np.std(M2)))

In [ ]:
mat = same_user_mat.copy()
np.fill_diagonal(mat, 1)   # remove the diagnoal from consideration
rows, cols = np.where(mat == 0)
M3 = M1[rows, cols]
print('Min: %.5f, Max: %.5f, Mean: %.5f, Std: %.5f' % (np.min(M3), np.max(M3), np.mean(M3), np.std(M3)))

**Check performance per user**

In [ ]:
user_set = sorted(set(user_of_playlists2))
#user_set

In [ ]:
#user_of_playlists2

In [ ]:
for u in user_set:
    uind = np.where(np.array(user_of_playlists2, dtype=np.object) == u)[0]
    uind_train = uind[uind < col_split]
    uind_test = uind[uind >= col_split]
    #print(uind)
    if len(uind_test) < 1: continue
    print('--------------------')
    print('USER:', u)
    print('#train: %d, #test: %d' % (len(uind_train), len(uind_test)))
    preds_test = [Y_pla[isnanmat[:, col], col] for col in uind_test]
    gts_test = [Y[isnanmat[:, col], col] for col in uind_test]
    print('Dev set:')
    eval_pl(gts_test, preds_test, useLoop=True)
    print('Training set:')
    eval_pl(Y[:, uind_train], Y_pla[:, uind_train])
    print()